In [18]:
import pandas as pd
import numpy as np
import math
import collections

In [19]:
# loading the data in a dataframe
cname=['satisfaction_level','last_evaluation','number_project','avg_monthly_hours','time_spend','work_accident','target','promotion','sales','salary']
file = 'train.csv'
df = pd.read_csv(file)
x = np.unique(df['left'], return_counts = True)

c_list = ['satisfaction_level','last_evaluation','number_project','average_montly_hours','time_spend_company','Work_accident','promotion_last_5years','sales','salary','left']
df1 = df[c_list]

# print(df1)
train_set = df1.sample(frac=0.8,random_state=200)
validate_set = df1.drop(train_set.index)
x = np.unique(train_set['left'], return_counts = True)
print(x)
c_list.remove('left')
# train_set

(array([0, 1]), array([6813, 2177]))


In [20]:
#### gini_index
def gini_index(df):
    total = df['left'].count()
    positive = df[df['left'] == 1]
    pos_c = positive['left'].count()
    neg_c = total - pos_c
    if pos_c == 0 or neg_c == 0 :
        return 0
    ent = - (math.pow(pos_c/total,2) 
                 + math.pow(neg_c/total,2))
    return ent
    
# entropy_(df1)

In [44]:
#Misclassification Rate
def miss_rate(df):
    total = df['left'].count()
    positive = df[df['left'] == 1]
    pos_c = positive['left'].count()
    neg_c = total - pos_c
    if pos_c == 0 or neg_c == 0 :
        return 0
    ent =  min((pos_c/total) 
                 ,(neg_c/total))
    return ent
    

In [45]:
def temp_f(data,a):
    return ([data[a].unique()])
def IG(data,attribute,entropy):
    isum = 0
    t = len(data.index)
    label = (data[attribute].unique()).tolist()
    for i in label:
        sdf = data[data[attribute] == i]
#         isum = isum + ((len(sdf.index)/t) * entropy_(sdf))
#         isum = isum + ((len(sdf.index)/t) * gini_index(sdf))
        isum = isum + ((len(sdf.index)/t) * miss_rate(sdf))
    return entropy - isum
# e = entropy_(df1)
# print(IG(df1,'salary',e))

In [46]:

def best_attribute(df,attribute):
#     e = entropy_(df)                     
#     e = gini_index(df)                         #gini
    e = miss_rate(df)
    attribute_list = {}
    for best in attribute :
        attribute_list[best] = IG(df , best , e)
#     m = -1
#     for i in attribute_list:
#         if attribute_list[i] > m:
#             m=attribute_list[i]
#             b = i
    b = max(attribute_list, key=attribute_list.get)
    return b

In [47]:
numeric_data = ['satisfaction_level','last_evaluation','number_project',
                'average_montly_hours','time_spend_company']
attribute_dict = {}

In [48]:
def convert(df,attr):
    mapp = {}
    check = []
    avg = []
    label = 0
    llist = []
    sortd = df.sort_values(attr)
    uv = sortd[attr].unique().tolist()
    for i in uv:
        temp = sortd[sortd[attr] == i]
        mapp[i] = np.unique(temp['left'])[np.argmax(np.unique(temp['left'],return_counts=True)[1])]
        
    prev = uv[0]
    h = mapp[uv[0]]
    for i in uv[1:len(uv)]:
        if mapp[i] == h:
            prev =i
            continue
        else :
            h = 1-h
            check.append(prev)
            prev = i
            check.append(i)
    for i in range(0,len(check),2):
        avg.append((check[i]+check[i+1])/2)
    attribute_dict[attr] = avg
    for i in df.index:
        attv = df.loc[i,attr]
        label = 0
        f = 0
        for a in avg:
            if attv < a:
                f = 1
                llist.append(label)
                break
            else :
                label += 1
        if f == 0:
            llist.append(label)
    
    df = df.drop(attr,axis=1)           ##
    new1 = pd.Series(llist)              ##
    df[attr] = new1.values
    return df
# convert(train_set,'satisfaction_level')

In [49]:
# convert train data to desirable range
for i in numeric_data:
    train_set = convert(train_set,i)

In [50]:
def tree(df , attribute_list , pd , root=None):
    if len(np.unique(df['left'])) <=1 :
        return {'predict': np.unique(df['left'])[0]}
    elif len(attribute_list) == 0:
        t = np.argmax(np.unique(df['left'], return_counts = True)[1])
        return {'predict': np.unique(df['left'])[t]}
    elif len(df) == 0:
        t = np.argmax(np.unique(pd['left'], return_counts = True)[1])
        return {'predict': np.unique(pd['left'])[t]}
    b = best_attribute(df,attribute_list[:])
    attribute_list.remove(b)
    # tree empty
    if root is None:
        root = {}
        root[b] = {}
    for i in df[b].unique():
        new_df = df[df[b]==i]
        root[b][i] = tree( new_df , attribute_list[:] , df)
    return root
# print (c_list)
root = tree(train_set,c_list[:],train_set)
print(root)

{'satisfaction_level': {3: {'time_spend_company': {0: {'average_montly_hours': {2: {'last_evaluation': {2: {'promotion_last_5years': {0: {'number_project': {1: {'Work_accident': {0: {'sales': {'support': {'salary': {'low': {'predict': 0}, 'medium': {'predict': 0}, 'high': {'predict': 0}}}, 'IT': {'predict': 0}, 'sales': {'salary': {'low': {'predict': 0}, 'medium': {'predict': 0}, 'high': {'predict': 0}}}, 'management': {'predict': 0}, 'technical': {'salary': {'medium': {'predict': 0}, 'low': {'predict': 0}, 'high': {'predict': 0}}}, 'marketing': {'salary': {'low': {'predict': 0}, 'high': {'predict': 0}, 'medium': {'predict': 0}}}, 'RandD': {'salary': {'high': {'predict': 0}, 'medium': {'predict': 0}, 'low': {'predict': 0}}}, 'accounting': {'predict': 0}, 'hr': {'salary': {'low': {'predict': 0}, 'medium': {'predict': 0}, 'high': {'predict': 0}}}, 'product_mng': {'predict': 0}}}, 1: {'sales': {'IT': {'predict': 0}, 'support': {'salary': {'low': {'predict': 0}, 'medium': {'predict': 0}, '

In [51]:
def prediction_convert(df,attr):
    avg = attribute_dict[attr]
    label = 0
    llist =[]
    for i in df.index:
        attv = df.loc[i,attr]
        label = 0
        f = 0
        for a in avg:
            if attv < a:
                f = 1
                llist.append(label)
                break
            else :
                label += 1
        if f == 0:
            llist.append(label)
    
    df = df.drop(attr,axis=1)           ##
    new1 = pd.Series(llist)              ##
    df[attr] = new1.values
    return df

In [52]:
for i in numeric_data:
    validate_set = prediction_convert(validate_set,i)

In [53]:
def compare(row,root):
    try:
        if list(root.keys())[0] == 'predict':
            return root['predict']
        else:
            value = row[list(root.keys())[0]]
            return compare(row,root[list(root.keys())[0]][value])
    except:
        return 0         

In [54]:
prediction = []  
def test(root,df):
    for index,row in df.iterrows():
        prediction.append(compare(row,root))
test(root,validate_set)

In [55]:
actual = validate_set['left'].tolist()

def final_prediction_list():
    prediction_list= OrderedDict()
    tp,fp,tn,fn = 0,0,0,0
    for i in range(len(actual)):
        if prediction[i] == actual[i]:
            if prediction[i] == 1:
                tp += 1
            elif prediction[i] == 0:
                tn += 1
        else:
            if actual[i] == 1:
                fn += 1
            elif actual[i] == 0:
                fp += 1
    prediction_list['True Positive'] = tp
    prediction_list['False Positive'] = fp
    prediction_list['True Negative'] = tn
    prediction_list['False Negative'] = fn
    accuracy = (tp+tn)/len(actual)
    precision = (tp)/(tp+fp)
    recall = (tp)/(tp+fn)
    f1_score = 2/(1/recall + 1/precision)
    prediction_list['Accuracy'] = accuracy
    prediction_list['Precision'] = precision
    prediction_list['Recall'] = recall
    prediction_list['F1_Score'] = f1_score
    pp = pprint.PrettyPrinter(indent=6)
    for i, (key, value) in enumerate(prediction_list.items()):
        print(key + ":: ", value)
        print("")
final_prediction_list()

455 1703 47 43
0.9599644128113879 0.9063745019920318 0.9136546184738956 0.9099999999999999
